In [1]:
import gym
env = gym.make("CartPole-v0")

/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:505: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1` with the environment ID `CartPole-v1`.
  logger.warn(


In [2]:
env.reset() # initial observation

for i in range(20):

  # env.action_space.sample() produces either 0 (left) or 1 (right).
  observation, reward, done, info = env.step(env.action_space.sample())

  print("step", i, observation, reward, done, info)

env.close()

step 0 [ 0.04526065  0.23341721 -0.02059129 -0.3466628 ] 1.0 False {}
step 1 [ 0.049929    0.4288259  -0.02752455 -0.6457671 ] 1.0 False {}
step 2 [ 0.05850552  0.2340981  -0.04043989 -0.36187723] 1.0 False {}
step 3 [ 0.06318748  0.03957358 -0.04767743 -0.0822151 ] 1.0 False {}
step 4 [ 0.06397895  0.23534541 -0.04932174 -0.389551  ] 1.0 False {}
step 5 [ 0.06868586  0.04095693 -0.05711276 -0.11281758] 1.0 False {}
step 6 [ 0.069505    0.23684877 -0.05936911 -0.42295825] 1.0 False {}
step 7 [ 0.07424197  0.43275934 -0.06782828 -0.73375064] 1.0 False {}
step 8 [ 0.08289716  0.6287496  -0.08250329 -1.0469862 ] 1.0 False {}
step 9 [ 0.09547215  0.8248638  -0.10344301 -1.3643854 ] 1.0 False {}
step 10 [ 0.11196943  0.6311784  -0.13073072 -1.1057683 ] 1.0 False {}
step 11 [ 0.124593    0.43799472 -0.15284608 -0.85679096] 1.0 False {}
step 12 [ 0.13335289  0.63483185 -0.16998191 -1.1933664 ] 1.0 False {}
step 13 [ 0.14604953  0.83169764 -0.19384924 -1.5341436 ] 1.0 False {}
step 14 [ 0.1626

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/envs/classic_control/cartpole.py:163: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  logger.warn(


**Dependencies for video display**

In [3]:
!apt-get install -y xvfb x11-utils
!pip install pyvirtualdisplay==0.2.*

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
x11-utils is already the newest version (7.7+5build2).
xvfb is already the newest version (2:21.1.4-2ubuntu1.7~22.04.12).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [4]:
from pyvirtualdisplay import Display
display = Display(visible=False, size=(1400, 900))
_ = display.start()

In [5]:
from gym.wrappers.monitoring.video_recorder import VideoRecorder
before_training = "before_training.mp4"

video = VideoRecorder(env, before_training)
# returns an initial observation
env.reset()
for i in range(200):
  env.render()
  video.capture_frame()
  # env.action_space.sample() produces either 0 (left) or 1 (right).
  observation, reward, done, info = env.step(env.action_space.sample())
  # Not printing this time
  #print("step", i, observation, reward, done, info)

video.close()
env.close()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/wrappers/monitoring/video_recorder.py:115: DeprecationWarning: WARN: `env.metadata["video.frames_per_second"] is marked as deprecated and will be replaced with `env.metadata["render_fps"]` see https://github.com/openai/gym/pull/2654 for more details
  logger.deprecation(
/usr/local/lib/python3.10/dist-packages/pygame/pkgdata.py:27: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarn

**Helper function to display video of untrained cartpole**

In [6]:
from base64 import b64encode
def render_mp4(videopath: str) -> str:
  """
  Gets a string containing a b4-encoded version of the MP4 video
  at the specified path.
  """
  mp4 = open(videopath, 'rb').read()
  base64_encoded_mp4 = b64encode(mp4).decode()
  return f'<video width=400 controls><source src="data:video/mp4;' \
         f'base64,{base64_encoded_mp4}" type="video/mp4"></video>'


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
from IPython.display import HTML
html = render_mp4(before_training)
HTML(html)

In [8]:
!pip install 'ray[rllib]'==2.0

In [9]:
!pip install gym[classic_control]


In [ ]:
import ray
from ray.rllib.agents.ppo import PPOTrainer
from ray.tune import run

config = {
    "env": "CartPole-v0",
    "framework": "torch",  # Use "tf" for TensorFlow
    "model": {
        "fcnet_hiddens": [32],
        "fcnet_activation": "tanh",
    },
}
stop = {
    "episode_reward_mean": 195,
    "training_iteration": 100,  # Stop after 100 iterations, even if the reward isn't met
}

ray.shutdown()
ray.init(
    num_cpus=1,  # Adjust to available resources
    include_dashboard=False,
    ignore_reinit_error=True,
    log_to_driver=False,
)

# Execute training
results = run(
    "PPO",
    config=config,
    stop=stop,
    checkpoint_at_end=True,
    verbose=1,
)

# Access the best trial or results
best_result = results.get_best_result(metric="episode_reward_mean", mode="max")
print(f"Best trial config: {best_result.config}")
print(f"Best trial final reward: {best_result.metrics['episode_reward_mean']}")


In [12]:
# restore a trainer from the last checkpoint
trial = results.get_best_logdir("episode_reward_mean", "max")
checkpoint = results.get_best_checkpoint(
  trial,
  "training_iteration",
  "max",
)
trainer = PPOTrainer(config=config)
trainer.restore(checkpoint)

NameError: name 'results' is not defined

In [ ]:
after_training = "after_training.mp4"
after_video = VideoRecorder(env, after_training)
observation = env.reset()
done = False
while not done:
  env.render()
  after_video.capture_frame()
  action = trainer.compute_action(observation)
  observation, reward, done, info = env.step(action)
after_video.close()
env.close()
# You should get a video similar to the one below.
html = render_mp4(after_training)
HTML(html)
